In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
os.chdir("/home/yw699/codes/LLM-Hallu")
sys.path.append(os.path.abspath("src"))

## 读取配置文件

In [ ]:
import yaml
with open("configs/experiment_config1.yaml", "r") as file:
    config = yaml.safe_load(file)

## Dataset加载与格式化

In [ ]:
from dataset import Dataset
dataset_loader = Dataset(config)
train_dataset, validation_dataset = dataset_loader.load_data()

print(validation_dataset[:1])  

## Prompt Engineer

In [ ]:
from prompt_engineer import PromptGenerator
promptgenerator = PromptGenerator(config,validation_dataset)

In [ ]:
prompt = promptgenerator.generate_prompt_by_id(1)
len(promptgenerator.get_unused_indices())
print(prompt)

In [ ]:
prompts =promptgenerator.generate_prompts_by_count(5)
len(promptgenerator.get_unused_indices())

## model

In [ ]:
from models import HuggingfaceModel
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
huggingface_model = HuggingfaceModel(config)

In [ ]:
#temperature=1.0,return_full=False
output_text = huggingface_model.predict(prompt)
print("Generated Text > \n", output_text)

In [ ]:
import math

user_tag = "USER:"
assistant_tag = "ASSISTANT:"
input = "What is the capital of France?"
v = huggingface_model.get_p_true(input,"Washington")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Paris")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Beijing")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

v = huggingface_model.get_p_true(input,"\n")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

In [ ]:
input_text = prompts[1]

output_text,log = huggingface_model.predict(input_text)
print("<Generated Text>:", output_text)
print(log)

In [ ]:
from samplers import Sampler

sampler = Sampler(prompt,huggingface_model,sample_method="simple")

In [ ]:
sampler.sample(8)
sampler.show_all_result()